# 前向自动微分
前向自动微分是指从计算图的起点开始，沿着计算图边的方向依次向前计算，最终达到计算图的终点。根据自变量的值计算出计算图中每个节点的值及其导数值，并保留中间结果，一直得到整个函数的值及其导数值。

步骤：
1. 将程序分解为一系列已知微分规则的基础表达式组合
2. 根据已知微分规则给出各基本表达式的微分结果。  操作符重载的方法是指在负载运算过程中，根据已知微分规则给出各基础表达式的微分结果
3. 根据链式法则计算出最终微分结果

# 具体实现
以计算$f(x1, x2) = ln(x1) + x1 \cdot x2 - sin(x2)$ 为例
1. 首先导入 `Numpy` 库

In [7]:
import numpy as np

2. 创建 `ADTangent` 类，

3. 操作符重写基本运算

In [8]:
class ADTangent:
    # 自变量 x，对自变量进行求导得到的 dx
    def __init__(self, x, dx):
        self.x = x
        self.dx = dx
    
    # 重载 str 是为了方便打印的时候，看到输入的值和求导后的值
    def __str__(self):
        context = f"value is {self.x:.4f}, grad is {self.dx}"
        return context
    
    def __add__(self, other):
        if isinstance(other, ADTangent):
            x = self.x + other.x
            dx = self.dx + other.dx
        elif isinstance(other, float):
            x = self.x + other
            dx = self.dx
        else:
            raise ValueError("input must be ADTengent or float!")
        return ADTangent(x, dx)
    
    def __sub__(self, other):
        if isinstance(other, ADTangent):
            x = self.x - other.x
            dx = self.dx - other.dx
        elif isinstance(other, float):
            x = self.x - other
            dx = self.dx
        else:
            return NotImplementedError
        return ADTangent(x, dx)

    def __mul__(self, other):
        if isinstance(other, ADTangent):
            x = self.x * other.x
            dx = self.x * other.dx + self.dx * other.x
        elif isinstance(other, float):
            x = self.x * other
            dx = self.dx * other
        else:
            return NotImplementedError
        return ADTangent(x, dx)

    def log(self):
        x = np.log(self.x)
        dx = 1 / self.x * self.dx
        return ADTangent(x, dx)

    def sin(self):
        x = np.sin(self.x)
        dx = self.dx * np.cos(self.x)
        return ADTangent(x, dx)
    

4. 计算当 $x1=2,x2=5$ 时 $f$ 关于自变量 $x1$ 的导数，因此在数据初始化时，将自变量 $x1$ 的 $dx$ 设置为1，$x2$ 的 $dx$ 设置为0.

In [9]:
x1 = ADTangent(x=2.0, dx=1)
x2 = ADTangent(x=5.0, dx=0)
f = ADTangent.log(x1) + x1 * x2 - ADTangent.sin(x2)
print(f)

value is 11.6521, grad is 5.5


## PyTorch 实现结果

In [10]:
import torch
from torch.autograd import Variable

x = Variable(torch.Tensor([2.]), requires_grad=True)
y = Variable(torch.Tensor([5.]), requires_grad=True)
f = torch.log(x) + x * y - torch.sin(y)
f.backward()

print(f)
print(x.grad)
print(y.grad)

tensor([11.6521], grad_fn=<SubBackward0>)
tensor([5.5000])
tensor([1.7163])
